<a href="https://colab.research.google.com/github/srirakshareddy/Datamining-on-retail-data/blob/main/Sriraksha_datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_excel("Online Retail.xlsx")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# discarding rows with no customer IDs

data.dropna(subset=['CustomerID'],how='all',inplace=True)
data.shape

(406829, 8)

In [ ]:
# discarding cancelled orders

data = data[data['Quantity']>0]
data.shape

(397924, 8)

In [ ]:
# date column only contains the date of the invoices
data['date'] = pd.DatetimeIndex(data['InvoiceDate']).date

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


In [ ]:
# group by customers by max date

recency_df = data.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18
1,12347.0,2011-12-07
2,12348.0,2011-09-25
3,12349.0,2011-11-21
4,12350.0,2011-02-02


In [ ]:
data['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [ ]:
import datetime as dt
now = dt.date(2011,12,9)
print(now)

2011-12-09


In [ ]:
# get recency value

recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x: (now - x).days)
recency_df.head()

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310


In [ ]:
recency_df.drop('LastPurchaseDate',axis=1,inplace=True)

In [ ]:
# droping duplicates
retail_data = data
retail_data.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)

#calculate frequency of purchases
frequency_df = retail_data.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

,CustomerID,Frequency
0,12346.0,1
1,12347.0,7
2,12348.0,4
3,12349.0,1
4,12350.0,1


In [ ]:
data['TotalCost'] = data['Quantity'] * data['UnitPrice']
monetary_df = data.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,163.16
2,12348.0,331.36
3,12349.0,15.00
4,12350.0,25.20


In [ ]:
temp_df = recency_df.merge(frequency_df,on='CustomerID')
temp_df.head()

,CustomerID,Recency,Frequency
0,12346.0,325,1
1,12347.0,2,7
2,12348.0,75,4
3,12349.0,18,1
4,12350.0,310,1


In [ ]:
# merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='CustomerID')

#use CustomerID as index
rfm_df.set_index('CustomerID',inplace=True)

#check the head
rfm_df.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,1,77183.60
12347.0,2,7,163.16
12348.0,75,4,331.36
12349.0,18,1,15.00
12350.0,310,1,25.20


In [ ]:
# declare quantiles

quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,17.0,1.0,17.700
0.50,50.0,2.0,47.000
0.75,141.5,5.0,130.045


In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
    
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,1,1,4
12347.0,2,7,163.16,4,4,4
12348.0,75,4,331.36,2,3,4
12349.0,18,1,15.00,3,1,1
12350.0,310,1,25.20,1,1,2


In [ ]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,1,1,4,114
12347.0,2,7,163.16,4,4,4,444
12348.0,75,4,331.36,2,3,4,234
12349.0,18,1,15.00,3,1,1,311
12350.0,310,1,25.20,1,1,2,112


In [ ]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,60,42053.60,4,4,4,444
17949.0,1,45,29999.69,4,4,4,444
17450.0,8,46,26768.97,4,4,4,444
14646.0,1,74,19045.06,4,4,4,444
16013.0,3,47,17895.28,4,4,4,444


In [ ]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Profitable Customers: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Verge of churning: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))

Best Customers:  401
Loyal Customers:  872
Profitable Customers:  1085
Verge of churning:  66
Lost Customers:  11


In [ ]:
# best customers:
rfm_segmentation[rfm_segmentation['RFMScore']=='444']

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12347.0,2,7,163.16,4,4,4,444
12362.0,3,10,215.13,4,4,4,444
12388.0,15,6,140.52,4,4,4,444
12417.0,3,9,141.40,4,4,4,444
12433.0,0,7,198.96,4,4,4,444
...,...,...,...,...,...,...,...
18225.0,3,12,295.11,4,4,4,444
18229.0,11,20,1542.96,4,4,4,444
18230.0,9,7,279.72,4,4,4,444


In [ ]:
# verge of churning
rfm_segmentation[rfm_segmentation['RFMScore']=='244']

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12457.0,66,7,814.00,2,4,4,244
12637.0,67,8,213.62,2,4,4,244
12643.0,128,6,237.36,2,4,4,244
12744.0,51,7,6449.45,2,4,4,244
12853.0,134,6,202.20,2,4,4,244
...,...,...,...,...,...,...,...
17863.0,59,6,183.88,2,4,4,244
17894.0,71,8,183.74,2,4,4,244
18008.0,70,9,1833.84,2,4,4,244


In [ ]:
# profitable customers
rfm_segmentation[rfm_segmentation['M_Quartile']==4]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,1,1,4,114
12347.0,2,7,163.16,4,4,4,444
12348.0,75,4,331.36,2,3,4,234
12352.0,36,8,536.30,3,4,4,344
12356.0,22,3,153.90,3,3,4,334
...,...,...,...,...,...,...,...
18246.0,23,2,407.40,3,2,4,324
18251.0,87,1,252.00,2,1,4,214
18257.0,39,8,153.43,3,4,4,344


In [ ]:
# lost customers
rfm_segmentation[rfm_segmentation['RFMScore']=='111']

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12405.0,148,1,12.48,1,1,1,111
12426.0,194,1,17.70,1,1,1,111
12441.0,366,1,10.50,1,1,1,111
12501.0,336,1,8.50,1,1,1,111
12506.0,232,1,7.95,1,1,1,111
...,...,...,...,...,...,...,...
18213.0,236,1,15.00,1,1,1,111
18224.0,263,1,10.20,1,1,1,111
18227.0,217,1,15.00,1,1,1,111


In [ ]:
# customers that must be retained
rfm_segmentation[(rfm_segmentation['RFMScore']=='444') | (rfm_segmentation['M_Quartile']==4)]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,1,1,4,114
12347.0,2,7,163.16,4,4,4,444
12348.0,75,4,331.36,2,3,4,234
12352.0,36,8,536.30,3,4,4,344
12356.0,22,3,153.90,3,3,4,334
...,...,...,...,...,...,...,...
18246.0,23,2,407.40,3,2,4,324
18251.0,87,1,252.00,2,1,4,214
18257.0,39,8,153.43,3,4,4,344


In [ ]:
# loyal customers
rfm_segmentation[rfm_segmentation['F_Quartile']==4]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12347.0,2,7,163.16,4,4,4,444
12352.0,36,8,536.30,3,4,4,344
12362.0,3,10,215.13,4,4,4,444
12388.0,15,6,140.52,4,4,4,444
12395.0,19,12,215.43,3,4,4,344
...,...,...,...,...,...,...,...
18245.0,7,7,132.86,4,4,4,444
18257.0,39,8,153.43,3,4,4,344
18260.0,172,6,150.57,1,4,4,144


In [ ]:
# customers that are most likely to respond to a current campaign
rfm_segmentation[(rfm_segmentation['F_Quartile']==4) | (rfm_segmentation['RFMScore']=='444')]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12347.0,2,7,163.16,4,4,4,444
12352.0,36,8,536.30,3,4,4,344
12362.0,3,10,215.13,4,4,4,444
12388.0,15,6,140.52,4,4,4,444
12395.0,19,12,215.43,3,4,4,344
...,...,...,...,...,...,...,...
18245.0,7,7,132.86,4,4,4,444
18257.0,39,8,153.43,3,4,4,344
18260.0,172,6,150.57,1,4,4,144
